# COVID-19 Dataset

* In this project, I am showing different ways to collect COVID-19 dataset.
## For theoratical undestanding refer my medium page [Vivek Dhameliya on medium](https://medium.com/@dhameliyavivek99/data-science-for-covid-19-part-1-6687a62f9d63)


## 1. Cloning from the GitHub
 * John Hopkins (GITHUB) https://github.com/CSSEGISandData/COVID-19.git
 * I already cloned the above repositary via GIT BASH in my local drive where I want to save COVID-19 dataset

In [1]:
# importing python packages
import os
import pandas as pd
import subprocess
# set limit for displaying max amount of raws values for dataframe
pd.set_option('display.max_rows', 500)

In [2]:
# pulling data from github and storing in local drive
git_pull = subprocess.Popen( 'git pull' , 
                     cwd = os.path.dirname( 'C:/Users/dhame/ds_covid-19/data/raw/COVID-19/' ), 
                     shell = True, 
                     stdout = subprocess.PIPE, 
                     stderr = subprocess.PIPE )
(out, error) = git_pull.communicate()


print("Error : " + str(error)) 
print("out : " + str(out))

Error : b''
out : b'Already up to date.\n'


In [3]:
df_1= pd.read_csv('C:/Users/dhame/ds_covid-19/data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')

In [4]:
df_1.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,...,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20,5/20/20,5/21/20,5/22/20,5/23/20,5/24/20,5/25/20,5/26/20,5/27/20,5/28/20,5/29/20,5/30/20,5/31/20,6/1/20,6/2/20,6/3/20,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20,6/12/20,6/13/20,6/14/20,6/15/20,6/16/20,6/17/20,6/18/20,6/19/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,4,4,5,7,7,7,11,16,21,22,...,1092,1176,1279,1351,1463,1531,1703,1828,1939,2171,2335,2469,2704,2894,3224,3392,3563,3778,4033,4402,4687,4963,5226,5639,6053,6402,6664,7072,7653,8145,8676,9216,9998,10582,11173,11831,12456,13036,13659,14525,15205,15750,16509,17267,18054,18969,19551,20342,20917,21459,22142,22890,23546,24102,24766,25527,26310,26874,27532,27878
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,10,12,23,33,38,42,51,55,...,609,634,663,678,712,726,736,750,766,773,782,789,795,803,820,832,842,850,856,868,872,876,880,898,916,933,946,948,949,964,969,981,989,998,1004,1029,1050,1076,1099,1122,1137,1143,1164,1184,1197,1212,1232,1246,1263,1299,1341,1385,1416,1464,1521,1590,1672,1722,1788,1838
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,3,5,12,12,17,17,19,20,20,20,24,26,37,48,54,60,...,2811,2910,3007,3127,3256,3382,3517,3649,3848,4006,4154,4295,4474,4648,4838,4997,5182,5369,5558,5723,5891,6067,6253,6442,6629,6821,7019,7201,7377,7542,7728,7918,8113,8306,8503,8697,8857,8997,9134,9267,9394,9513,9626,9733,9831,9935,10050,10154,10265,10382,10484,10589,10698,10810,10919,11031,11147,11268,11385,11504
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,39,...,717,723,723,731,738,738,743,743,743,745,745,747,748,750,751,751,752,752,754,755,755,758,760,761,761,761,761,761,761,762,762,762,762,762,763,763,763,763,764,764,764,765,844,851,852,852,852,852,852,852,852,852,853,853,853,853,854,854,855,855
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,24,25,25,25,25,26,27,27,27,27,30,35,35,35,36,36,36,43,43,45,45,45,45,48,48,48,48,50,52,52,58,60,61,69,70,70,71,74,81,84,86,86,86,86,86,86,88,91,92,96,113,118,130,138,140,142,148,155,166,172


## 2.1 Webscrapping on RKI webpage
* https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html

In [5]:
#python packages
import requests
from bs4 import BeautifulSoup

In [6]:
# create a page by giving URL of RKI Fallzahlen
page = requests.get("https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html")

In [7]:
# parseing the html link
soup = BeautifulSoup(page.content, 'html.parser')

In [8]:
#find the table, and this works if only one table exists
html_table=soup.find('table')

In [9]:
# use the table id 'tr' to extract data in forms of rows starting
all_rows=html_table.find_all('tr')

In [10]:
# create a empty list and than strip each text from rows data
data_list=[]
for pos,rows in enumerate(all_rows):
    col_list=[each_col.get_text(strip=True) for each_col in rows.find_all('td')] #td for data element
    data_list.append(col_list)

In [11]:
pd_daily_status=pd.DataFrame(data_list).dropna().rename(columns={0:'state',
                                                       1:'cases',
                                                       2:'changes',
                                                       3:'cases_per_100k',
                                                       4:'fatal',
                                                       5:'comment'})

In [12]:
pd_daily_status.head()

,state,cases,changes,cases_per_100k,fatal,comment
2,Baden-Württem­berg,35.272,+36,139,"1,3",1.824
3,Bayern,47.827,+28,217,"1,7",2.566
4,Berlin,7.733,+130,452,"12,1",211
5,Branden­burg,3.383,+9,45,"1,8",163
6,Bremen,1.636,+10,55,"8,1",49


## 2.2 Webscrapping on Worldometer wegpage
* https://www.worldometers.info/coronavirus/

In [13]:
# python package
import csv

In [14]:
# giving URL address
URL = 'https://www.worldometers.info/coronavirus/'

In [15]:
# use requests to get the html at given URL
source = requests.get(URL).text

In [16]:
# Now use BeautifulSoup object on the html
soup = BeautifulSoup(source,'lxml')

In [17]:
# get the table with all of the countries/stats
table_1 = soup.find('table',id="main_table_countries_today")

In [18]:
# use the table id 'th' to extract data in forms of rows starting
table_header = table_1.find('thead').find_all('th')

In [34]:
# open a CSV file to write all the the data to
with open('COVID_19_from_worldometer.csv','w',newline='') as f:
    # csv writer object
    writer = csv.writer(f)

    # empty header list will store the raw data from the header
    # empty new_list list will store the data after parsing it to characters
    header = []
    new_list = []

    # get the data from the header and add it to the list
    for th in table_header:
        header.append(th.text)

    # strip the new lines and add it to the new list
    for element in header:
        new_list.append(element.strip())

    # remove all new lines / weird unicode chars
    for i,data in enumerate(new_list):
        new_list[i] = data.replace('\xa0', ' ')
        new_list[i] = data.replace('\n',' ')

    # write the row to the csv
    writer.writerow(new_list)

    # go through each row in the table and 
    for row in table_1.find_all('tr'):

        # csvRow list used to hold the data in each row
        csvRow = []

        # get the data and add it to the list
        for data in row.find_all('td'):
            csvRow.append(data.text)
        
        # remove all \n from the data
        for i in range(len(csvRow)):
            csvRow[i] = csvRow[i].replace('\n','')

        # write the csvRow data to the csvfile
        writer.writerow(csvRow)

# close file
f.close()

## 3.1 REST API 
* REST API services to retreive data from https://npgeo-corona-npgeo-de.hub.arcgis.com/

In [20]:
#python package
import json

In [21]:
# creating data request for Germany country
data=requests.get('https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/Coronaf%C3%A4lle_in_den_Bundesl%C3%A4ndern/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json')

In [22]:
# use json package to load data from called REST API
json_object=json.loads(data.content) 

In [23]:
#checking data_type of Json_object
type(json_object)

dict

In [24]:
# getting keys of json file
json_object.keys()

dict_keys(['objectIdFieldName', 'uniqueIdField', 'globalIdFieldName', 'geometryProperties', 'geometryType', 'spatialReference', 'fields', 'features'])

In [25]:
# create empty list name full_list and than append data from json_object
full_list=[]
for pos,each_dict in enumerate (json_object['features'][:]):
    full_list.append(each_dict['attributes'])

In [26]:
# convert full_list to pandas dataframe
pd_full_list=pd.DataFrame(full_list)
pd_full_list.head()

,AGS_TXT,Aktualisierung,Death,Fallzahl,GlobalID,LAN_ew_AGS,LAN_ew_BEZ,LAN_ew_EWZ,LAN_ew_GEN,OBJECTID,OBJECTID_1,Shape__Area,Shape__Length,faelle_100000_EW
0,01,1592604000000,152,3133,fc5ba936-c95c-432c-8a33-9eb2f30b660f,01,Land,2896712,Schleswig-Holstein,15,1,4.573731e+10,2.881496e+06,108.157111
1,02,1592604000000,259,5160,0f3e860c-5181-4d3f-a421-1d51f50315ea,02,Freie und Hansestadt,1841179,Hamburg,6,2,2.089396e+09,4.188002e+05,280.255206
2,03,1592604000000,618,13136,3fd77024-c29b-4843-9be8-682ad48e60c9,03,Land,7982448,Niedersachsen,9,3,1.299836e+11,4.008988e+06,164.561047
3,04,1592604000000,49,1636,4132268b-54de-4327-ac1e-760e915112f1,04,Freie Hansestadt,682986,Bremen,5,4,1.119157e+09,3.357177e+05,239.536389
4,05,1592604000000,1663,40372,561d658f-3ee5-46e3-bc95-3528c6558ab9,05,Land,17932651,Nordrhein-Westfalen,10,5,8.782936e+10,2.648673e+06,225.131242


In [27]:
# save dataframe to local drive in CSV format
pd_full_list.to_csv('C:/Users/dhame/ds_covid-19/data/raw/NPGEO/GER_state_data.csv',sep=';')

In [28]:
pd_full_list.describe()

,Aktualisierung,Death,Fallzahl,LAN_ew_EWZ,OBJECTID,OBJECTID_1,Shape__Area,Shape__Length,faelle_100000_EW
count,1.600000e+01,16.000000,16.000000,1.600000e+01,16.000000,16.000000,1.600000e+01,1.600000e+01,16.000000
mean,1.592604e+12,555.187500,11820.937500,5.188701e+06,8.500000,8.500000,5.672418e+10,2.142056e+06,192.587329
std,0.000000e+00,761.924075,15097.334045,4.936479e+06,4.760952,4.760952,4.556489e+10,1.258959e+06,87.655148
min,1.592604e+12,20.000000,791.000000,6.829860e+05,1.000000,1.000000,1.119157e+09,3.357177e+05,49.140354
25%,1.592604e+12,160.250000,3043.750000,2.067654e+06,4.750000,4.750000,3.202908e+10,1.471492e+06,134.162531
50%,1.592604e+12,216.000000,5284.000000,3.270769e+06,8.500000,8.500000,5.009895e+10,2.098262e+06,168.730204
75%,1.592604e+12,529.500000,11201.000000,6.694969e+06,12.250000,12.250000,8.023081e+10,2.706878e+06,249.716093
max,1.592604e+12,2566.000000,47827.000000,1.793265e+07,16.000000,16.000000,1.634855e+11,4.131181e+06,365.741534


## 3.2 Additional API Access via REST Dienst, i.e INDIA Dataset
* example of REST confirm interface (Important!!!: Registration is required)
* https://smartable.ai/

In [29]:
# I want data for INDIA, there are other choices also available
url_endpoint = 'https://api.smartable.ai/coronavirus/stats/IN'

headers = {
    'Cache-Control': 'mo-cache',
    'Subscription-Key': '22dc8d09733243328bacc2047f1c6f23'
}
response = requests.get(url_endpoint, headers=headers)

In [30]:
print(response)

<Response [200]>


In [31]:
# Check what response content
response.content

b'{\n  "location": {\n    "long": 78.0,\n    "countryOrRegion": "India",\n    "provinceOrState": null,\n    "county": null,\n    "isoCode": "IN",\n    "lat": 21.0\n  },\n  "updatedDateTime": "2020-06-20T02:29:22.237926Z",\n  "stats": {\n    "totalConfirmedCases": 395812,\n    "newlyConfirmedCases": 14327,\n    "totalDeaths": 12970,\n    "newDeaths": 365,\n    "totalRecoveredCases": 214206,\n    "newlyRecoveredCases": 9023,\n    "history": [\n      {\n        "date": "2020-01-22T00:00:00",\n        "confirmed": 0,\n        "deaths": 0,\n        "recovered": 0\n      },\n      {\n        "date": "2020-01-23T00:00:00",\n        "confirmed": 0,\n        "deaths": 0,\n        "recovered": 0\n      },\n      {\n        "date": "2020-01-24T00:00:00",\n        "confirmed": 0,\n        "deaths": 0,\n        "recovered": 0\n      },\n      {\n        "date": "2020-01-25T00:00:00",\n        "confirmed": 0,\n        "deaths": 0,\n        "recovered": 0\n      },\n      {\n        "date": "2020-01-

In [32]:
IN_dict = json.loads(response.content) # loading strings for India dataset and dump into JSON file with .txt format
with open ('C:/Users/dhame/ds_covid-19/data/raw/IN_data.txt','w') as outfile:
    json.dump(IN_dict, outfile,indent=2)

In [33]:
# put all dictionary type data for INDIA into dataframe
df_4 = pd.DataFrame(IN_dict)
df_4.head()

,location,updatedDateTime,stats
breakdowns,NaN,2020-06-20T02:29:22.237926Z,"[{'location': {'long': 78.0, 'countryOrRegion'..."
countryOrRegion,India,2020-06-20T02:29:22.237926Z,NaN
county,None,2020-06-20T02:29:22.237926Z,NaN
history,NaN,2020-06-20T02:29:22.237926Z,"[{'date': '2020-01-22T00:00:00', 'confirmed': ..."
isoCode,IN,2020-06-20T02:29:22.237926Z,NaN
